In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import os
import mra
from mra import *
import models
from models import *

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [18]:
def make_dataset(X, Y, f):
    x = tf.data.Dataset.from_tensor_slices(X)
    y = tf.data.Dataset.from_tensor_slices(Y)
    x = x.map(f)
    xy = tf.data.Dataset.zip((x, y))
    xy = xy.shuffle(10000)
    return xy

xy = pd.read_csv("../datasets/jannis.csv",delimiter=',').to_numpy()
np.random.shuffle(xy)
x = xy[:,1:]
y = xy[:,0]

x = (x - np.min(x, axis=0))/(np.max(x, axis=0) - np.min(x, axis=0))
y = y.reshape((len(x), 1)).astype("float32")
train_ds = make_dataset(x, y, lambda t: t)

2022-03-30 10:08:09.241309: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-03-30 10:08:09.241419: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: xgpd0
2022-03-30 10:08:09.241454: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: xgpd0
2022-03-30 10:08:09.241698: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.103.1
2022-03-30 10:08:09.241776: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.103.1
2022-03-30 10:08:09.241795: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.103.1
2022-03-30 10:08:09.243110: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in per

In [20]:
main = {}

for split in [4]:
    print("split = {}".format(split))
    main[split] = mra(train_ds)
    make_f, make_g, make_generator = models.make_mlp(
        attr_num=54,
        class_num=4,
        split=split,
        fg_units=256,
        fg_act="relu",
        generator_units=[1024,512,256,128]
    )
    main[split].train(
        make_f,
        make_g,
        loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(),
        batch_size=32,
        epoch=5,
        lr=0.001,
        verbose=True, log_every=3000
    )

split = 4
Iteration 3000: Training loss: 0.8494
Iteration 6000: Training loss: 0.7835
Iteration 9000: Training loss: 0.7599
Iteration 12000: Training loss: 0.7498


In [22]:
log_attack = {}
for split in [4]:
    print("split level = {}".format(split))
    log_attack[split] = main[split].attack(
        attack_iter=100,
        make_generator=make_generator,
        input_noise="normal",
        input_z=True,
        model_leak=True,
        lr_x=0.0001,
        lr_f=0.000001,
        epoch=50,
        iter_x=40,
        iter_f=1,
        verbose=True, log_every=1
    )

split level = 4
Iteration 0001: RG-uniform: 0.1540, RG-normal: 0.1353, reconstruction validation: 0.0941
Iteration 0002: RG-uniform: 0.1516, RG-normal: 0.1318, reconstruction validation: 0.0891
Iteration 0003: RG-uniform: 0.1442, RG-normal: 0.1262, reconstruction validation: 0.1000
Iteration 0004: RG-uniform: 0.1548, RG-normal: 0.1278, reconstruction validation: 0.1102
Iteration 0005: RG-uniform: 0.1593, RG-normal: 0.1294, reconstruction validation: 0.1072
Iteration 0006: RG-uniform: 0.1540, RG-normal: 0.1401, reconstruction validation: 0.1136
Iteration 0007: RG-uniform: 0.1498, RG-normal: 0.1314, reconstruction validation: 0.1063
Iteration 0008: RG-uniform: 0.1518, RG-normal: 0.1353, reconstruction validation: 0.1097
Iteration 0009: RG-uniform: 0.1526, RG-normal: 0.1326, reconstruction validation: 0.1095
Iteration 0010: RG-uniform: 0.1491, RG-normal: 0.1331, reconstruction validation: 0.1028
Iteration 0011: RG-uniform: 0.1427, RG-normal: 0.1329, reconstruction validation: 0.1041
Itera

KeyboardInterrupt: 